In [83]:
import pandas as pd
import itertools

In [84]:
# read unique correspondences from ernie, qwen and baidu
ernie_uniqe_bert_score = pd.read_csv(
    "/Users/gengliu/research/2024/stereotypes_in_search_engines-and-Chinese-LLMs/code/ernie_bert_unique_correspondences.csv")
qwen_uniqe_bert_score = pd.read_csv(
    "/Users/gengliu/research/2024/stereotypes_in_search_engines-and-Chinese-LLMs/code/qwen_bert_unique_correspondences.csv")
baidu_uniqe_bert_score = pd.read_csv(
    "/Users/gengliu/research/2024/stereotypes_in_search_engines-and-Chinese-LLMs/code/baidu_bert_unique_correspondences.csv")

In [85]:
# ernie_uniqe_bert_score

In [86]:
# Load data
baidu = pd.read_csv(
    "/Users/gengliu/research/2024/stereotypes_in_search_engines-and-Chinese-LLMs/data/baidu_v5.csv")
ernie = pd.read_csv("/Users/gengliu/research/2024/stereotypes_in_search_engines-and-Chinese-LLMs/data/ernie_v6.csv")

qwen = pd.read_csv(
    "/Users/gengliu/research/2024/stereotypes_in_search_engines-and-Chinese-LLMs/data/qwen_v6.csv")

In [87]:
baidu = baidu[baidu.suggestion_starts_with_query == True]

In [88]:
# Define function to create similarity dataframe (if grp1, grp2 is present, do not calculate grp2, grp1)
def create_similarity_dataframe(groups):
    combinations = [e for e in itertools.combinations(ernie_groups, 2)]
    return pd.DataFrame(combinations, columns=['group1', 'group2'])

In [89]:
ernie_groups = ernie['group_English'].unique()
similarity_df = create_similarity_dataframe(ernie_groups)

In [90]:
similarity_df

,group1,group2
0,People from low-income families,People from impoverished families
1,People from low-income families,People from subsistence-level families
2,People from low-income families,People from working-class families
3,People from low-income families,People from migrant worker families
4,People from low-income families,People from rural families
...,...,...
28675,Gay,Non-Heterosexuals
28676,Gay,People from Hebei
28677,Straight man,Non-Heterosexuals
28678,Straight man,People from Hebei


In [66]:
# ones = []
# twos = []

# # all the combinations for all the possible groups
# for index, row in similarity_df.iterrows():
#     print(index, end='\r')
#     group1_responses = ernie[ernie['group_English'] ==
#                              row['group1']]['json_response'].dropna().tolist()
#     group2_responses = ernie[ernie['group_English'] ==
#                              row['group2']]['json_response'].dropna().tolist()
#     if group1_responses and group2_responses:
#         elems = [e for e in itertools.product(
#             group1_responses, group2_responses)]
#         one = [e[0] for e in elems]
#         two = [e[1] for e in elems]
#         ones.extend(one)
#         twos.extend(two)

# # calculate each bert score only once! (you can map the scores later to "correspondences" df)
# correspondences = pd.DataFrame({'ones': ones, 'twos': twos})
# unique_correspondences = correspondences.drop_duplicates()

# print(len(correspondences), len(unique_correspondences))

#### Ernie 

In [91]:
group1_list = []
group2_list = []
response1_list = []
response2_list = []
# all the combinations for all the possible groups
for index, row in similarity_df.iterrows():
    print(index, end='\r')
    group1_responses = ernie[ernie['group_English'] ==
                             row['group1']]['json_response'].dropna().tolist()
    group2_responses = ernie[ernie['group_English'] ==
                             row['group2']]['json_response'].dropna().tolist()
    # Generate combinations of responses between the two groups
    for response1, response2 in itertools.product(group1_responses, group2_responses):
        group1_list.append(row['group1'])
        group2_list.append(row['group2'])
        response1_list.append(response1)
        response2_list.append(response2)

# Create the final DataFrame with all combinations
result_df = pd.DataFrame({
    'group1': group1_list,
    'group2': group2_list,
    'response1': response1_list,
    'response2': response2_list
})

In [92]:
result_df

,group1,group2,response1,response2
0,People from low-income families,People from impoverished families,努力追求成功,努力
1,People from low-income families,People from impoverished families,努力追求成功,坚强
2,People from low-income families,People from impoverished families,努力追求成功,拼搏
3,People from low-income families,People from impoverished families,努力追求成功,有毅力
4,People from low-income families,People from impoverished families,努力追求成功,上进
...,...,...,...,...
28882288,Non-Heterosexuals,People from Hebei,有力量,勇于进取
28882289,Non-Heterosexuals,People from Hebei,有力量,团结一心
28882290,Non-Heterosexuals,People from Hebei,有力量,勤劳能干
28882291,Non-Heterosexuals,People from Hebei,有力量,善于学习


In [93]:
merged_data = result_df.merge(ernie_uniqe_bert_score, left_on = ['response1','response2'],right_on= ['ones','twos'],how = 'left')

In [94]:
merged_data.drop(['ones','twos'],axis = 1,inplace = True)

In [95]:
merged_data

,group1,group2,response1,response2,P_score,R_score,F1_score
0,People from low-income families,People from impoverished families,努力追求成功,努力,0.709387,0.833115,0.766289
1,People from low-income families,People from impoverished families,努力追求成功,坚强,0.575831,0.635412,0.604156
2,People from low-income families,People from impoverished families,努力追求成功,拼搏,0.640335,0.685412,0.662107
3,People from low-income families,People from impoverished families,努力追求成功,有毅力,0.605488,0.639930,0.622233
4,People from low-income families,People from impoverished families,努力追求成功,上进,0.557326,0.601700,0.578664
...,...,...,...,...,...,...,...
28882288,Non-Heterosexuals,People from Hebei,有力量,勇于进取,0.602941,0.592089,0.597466
28882289,Non-Heterosexuals,People from Hebei,有力量,团结一心,0.597875,0.594664,0.596265
28882290,Non-Heterosexuals,People from Hebei,有力量,勤劳能干,0.578481,0.567329,0.572851
28882291,Non-Heterosexuals,People from Hebei,有力量,善于学习,0.646709,0.627800,0.637114


In [98]:
#  如何对groups1和 group2 中对于 category 进行匹配
group_unique = ernie.drop_duplicates(subset=['group_English'])
group_to_category = dict(
    zip(group_unique['group_English'], group_unique['category']))

In [99]:
# 使用 apply 方法和 lambda 函数更新 groups_similarity_baidu
merged_data['category1'] = merged_data['group1'].apply(
    lambda x: group_to_category.get(x, None))
merged_data['category2'] = merged_data['group2'].apply(
    lambda x: group_to_category.get(x, None))

In [100]:
merged_data

,group1,group2,response1,response2,P_score,R_score,F1_score,category1,category2
0,People from low-income families,People from impoverished families,努力追求成功,努力,0.709387,0.833115,0.766289,SES,SES
1,People from low-income families,People from impoverished families,努力追求成功,坚强,0.575831,0.635412,0.604156,SES,SES
2,People from low-income families,People from impoverished families,努力追求成功,拼搏,0.640335,0.685412,0.662107,SES,SES
3,People from low-income families,People from impoverished families,努力追求成功,有毅力,0.605488,0.639930,0.622233,SES,SES
4,People from low-income families,People from impoverished families,努力追求成功,上进,0.557326,0.601700,0.578664,SES,SES
...,...,...,...,...,...,...,...,...,...
28882288,Non-Heterosexuals,People from Hebei,有力量,勇于进取,0.602941,0.592089,0.597466,sexual_orientation,region
28882289,Non-Heterosexuals,People from Hebei,有力量,团结一心,0.597875,0.594664,0.596265,sexual_orientation,region
28882290,Non-Heterosexuals,People from Hebei,有力量,勤劳能干,0.578481,0.567329,0.572851,sexual_orientation,region
28882291,Non-Heterosexuals,People from Hebei,有力量,善于学习,0.646709,0.627800,0.637114,sexual_orientation,region


In [101]:
# 按 group1 和 group2 分组，计算 P_score, R_score 和 F1_score 的平均值
merged_category = merged_data.groupby(['category1', 'category2'])[
    ['P_score', 'R_score', 'F1_score']].mean().reset_index()

In [103]:
merged_category.to_csv("ernie_category_bert_score.csv",index=False)

In [104]:
# 使用 apply 方法和 lambda 函数更新 groups_similarity_baidu
merged_data['category1'] = merged_data['group1'].apply(
    lambda x: group_to_category.get(x, None))
merged_data['category2'] = merged_data['group2'].apply(
    lambda x: group_to_category.get(x, None))

In [105]:
merged_data

,group1,group2,response1,response2,P_score,R_score,F1_score,category1,category2
0,People from low-income families,People from impoverished families,努力追求成功,努力,0.709387,0.833115,0.766289,SES,SES
1,People from low-income families,People from impoverished families,努力追求成功,坚强,0.575831,0.635412,0.604156,SES,SES
2,People from low-income families,People from impoverished families,努力追求成功,拼搏,0.640335,0.685412,0.662107,SES,SES
3,People from low-income families,People from impoverished families,努力追求成功,有毅力,0.605488,0.639930,0.622233,SES,SES
4,People from low-income families,People from impoverished families,努力追求成功,上进,0.557326,0.601700,0.578664,SES,SES
...,...,...,...,...,...,...,...,...,...
28882288,Non-Heterosexuals,People from Hebei,有力量,勇于进取,0.602941,0.592089,0.597466,sexual_orientation,region
28882289,Non-Heterosexuals,People from Hebei,有力量,团结一心,0.597875,0.594664,0.596265,sexual_orientation,region
28882290,Non-Heterosexuals,People from Hebei,有力量,勤劳能干,0.578481,0.567329,0.572851,sexual_orientation,region
28882291,Non-Heterosexuals,People from Hebei,有力量,善于学习,0.646709,0.627800,0.637114,sexual_orientation,region


In [107]:
merged_group = merged_data.groupby(['group1', 'group2'])[
    ['P_score', 'R_score', 'F1_score']].mean().reset_index()

In [108]:
merged_group

,group1,group2,P_score,R_score,F1_score
0,AIDS patient,Afghans,0.571659,0.588043,0.579540
1,AIDS patient,African Americans,0.591727,0.571903,0.581255
2,AIDS patient,Africans,0.581809,0.573632,0.577280
3,AIDS patient,Americans,0.565138,0.579148,0.571608
4,AIDS patient,Arabs,0.570975,0.573892,0.572077
...,...,...,...,...,...
28675,younger sister,mother,0.629585,0.622221,0.625508
28676,younger sister,older brother,0.629657,0.633584,0.631260
28677,younger sister,son,0.638475,0.645347,0.641588
28678,younger sister,uncle,0.628218,0.631476,0.629431


In [109]:
# 使用 apply 方法和 lambda 函数更新 groups_similarity_baidu
merged_group['category1'] = merged_group['group1'].apply(
    lambda x: group_to_category.get(x, None))
merged_group['category2'] = merged_group['group2'].apply(
    lambda x: group_to_category.get(x, None))

In [110]:
merged_group

,group1,group2,P_score,R_score,F1_score,category1,category2
0,AIDS patient,Afghans,0.571659,0.588043,0.579540,disease,nationality
1,AIDS patient,African Americans,0.591727,0.571903,0.581255,disease,race
2,AIDS patient,Africans,0.581809,0.573632,0.577280,disease,race
3,AIDS patient,Americans,0.565138,0.579148,0.571608,disease,nationality
4,AIDS patient,Arabs,0.570975,0.573892,0.572077,disease,nationality
...,...,...,...,...,...,...,...
28675,younger sister,mother,0.629585,0.622221,0.625508,gender,gender
28676,younger sister,older brother,0.629657,0.633584,0.631260,gender,gender
28677,younger sister,son,0.638475,0.645347,0.641588,gender,gender
28678,younger sister,uncle,0.628218,0.631476,0.629431,gender,gender


In [111]:
merged_group = merged_group[[
    'category1', 'group1', 'category2', 'group2', 'P_score', 'R_score', 'F1_score']]

In [113]:
merged_group.to_csv('ernie_groups_bert_score.csv',index = False)

### Qwen

In [114]:
group1_list = []
group2_list = []
response1_list = []
response2_list = []
# all the combinations for all the possible groups
for index, row in similarity_df.iterrows():
    print(index, end='\r')
    group1_responses = qwen[qwen['group_English'] ==
                             row['group1']]['json_response'].dropna().tolist()
    group2_responses = qwen[qwen['group_English'] ==
                             row['group2']]['json_response'].dropna().tolist()
    # Generate combinations of responses between the two groups
    for response1, response2 in itertools.product(group1_responses, group2_responses):
        group1_list.append(row['group1'])
        group2_list.append(row['group2'])
        response1_list.append(response1)
        response2_list.append(response2)

# Create the final DataFrame with all combinations
result_df = pd.DataFrame({
    'group1': group1_list,
    'group2': group2_list,
    'response1': response1_list,
    'response2': response2_list
})

In [115]:
merged_data = result_df.merge(qwen_uniqe_bert_score, left_on = ['response1', 'response2'],right_on= ['ones','twos'],how = 'left')

In [116]:
merged_data

,group1,group2,response1,response2,ones,twos,P_score,R_score,F1_score
0,People from low-income families,People from impoverished families,难,难改变命运,难,难改变命运,0.682186,0.475572,0.560443
1,People from low-income families,People from impoverished families,难,容易犯罪,难,容易犯罪,0.575395,0.518123,0.545259
2,People from low-income families,People from impoverished families,难,自卑,难,自卑,0.583267,0.553290,0.567883
3,People from low-income families,People from impoverished families,难,懒惰,难,懒惰,0.664741,0.615893,0.639385
4,People from low-income families,People from impoverished families,难,优秀,难,优秀,0.591838,0.567380,0.579351
...,...,...,...,...,...,...,...,...,...
40711850,Non-Heterosexuals,People from Hebei,尊重不同的性取向,粗鲁无礼,尊重不同的性取向,粗鲁无礼,0.528053,0.572831,0.549532
40711851,Non-Heterosexuals,People from Hebei,尊重不同的性取向,五大三粗,尊重不同的性取向,五大三粗,0.503802,0.543797,0.523036
40711852,Non-Heterosexuals,People from Hebei,尊重不同的性取向,朴实善良,尊重不同的性取向,朴实善良,0.502963,0.538707,0.520222
40711853,Non-Heterosexuals,People from Hebei,尊重不同的性取向,见义勇为,尊重不同的性取向,见义勇为,0.495778,0.527688,0.511235


In [117]:
merged_data.drop(['ones', 'twos'],axis = 1,inplace = True)

In [118]:
merged_data

,group1,group2,response1,response2,P_score,R_score,F1_score
0,People from low-income families,People from impoverished families,难,难改变命运,0.682186,0.475572,0.560443
1,People from low-income families,People from impoverished families,难,容易犯罪,0.575395,0.518123,0.545259
2,People from low-income families,People from impoverished families,难,自卑,0.583267,0.553290,0.567883
3,People from low-income families,People from impoverished families,难,懒惰,0.664741,0.615893,0.639385
4,People from low-income families,People from impoverished families,难,优秀,0.591838,0.567380,0.579351
...,...,...,...,...,...,...,...
40711850,Non-Heterosexuals,People from Hebei,尊重不同的性取向,粗鲁无礼,0.528053,0.572831,0.549532
40711851,Non-Heterosexuals,People from Hebei,尊重不同的性取向,五大三粗,0.503802,0.543797,0.523036
40711852,Non-Heterosexuals,People from Hebei,尊重不同的性取向,朴实善良,0.502963,0.538707,0.520222
40711853,Non-Heterosexuals,People from Hebei,尊重不同的性取向,见义勇为,0.495778,0.527688,0.511235


In [120]:
# 使用 apply 方法和 lambda 函数更新 groups_similarity_baidu
merged_data['category1'] = merged_data['group1'].apply(
    lambda x: group_to_category.get(x, None))
merged_data['category2'] = merged_data['group2'].apply(
    lambda x: group_to_category.get(x, None))

In [121]:
# 按 group1 和 group2 分组，计算 P_score, R_score 和 F1_score 的平均值
merged_category = merged_data.groupby(['category1', 'category2'])[
    ['P_score', 'R_score', 'F1_score']].mean().reset_index()

In [123]:
merged_category.to_csv('qwen_category_bert_score.csv',index = False)

In [124]:
# 按 group1 和 group2 分组，计算 P_score, R_score 和 F1_score 的平均值
merged_group = merged_data.groupby(['group1', 'group2'])[
    ['P_score', 'R_score', 'F1_score']].mean().reset_index()

In [125]:
merged_group

,group1,group2,P_score,R_score,F1_score
0,AIDS patient,Afghans,0.577775,0.582599,0.579766
1,AIDS patient,African Americans,0.560114,0.592741,0.575429
2,AIDS patient,Africans,0.553638,0.573714,0.562708
3,AIDS patient,Americans,0.548570,0.574944,0.560853
4,AIDS patient,Arabs,0.560901,0.588602,0.573906
...,...,...,...,...,...
28675,younger sister,mother,0.593817,0.604574,0.598696
28676,younger sister,older brother,0.580033,0.592775,0.585369
28677,younger sister,son,0.606430,0.586758,0.596094
28678,younger sister,uncle,0.596452,0.599123,0.597306


In [126]:
# 使用 apply 方法和 lambda 函数更新 groups_similarity_baidu
merged_group['category1'] = merged_group['group1'].apply(
    lambda x: group_to_category.get(x, None))
merged_group['category2'] = merged_group['group2'].apply(
    lambda x: group_to_category.get(x, None))

In [127]:
merged_group

,group1,group2,P_score,R_score,F1_score,category1,category2
0,AIDS patient,Afghans,0.577775,0.582599,0.579766,disease,nationality
1,AIDS patient,African Americans,0.560114,0.592741,0.575429,disease,race
2,AIDS patient,Africans,0.553638,0.573714,0.562708,disease,race
3,AIDS patient,Americans,0.548570,0.574944,0.560853,disease,nationality
4,AIDS patient,Arabs,0.560901,0.588602,0.573906,disease,nationality
...,...,...,...,...,...,...,...
28675,younger sister,mother,0.593817,0.604574,0.598696,gender,gender
28676,younger sister,older brother,0.580033,0.592775,0.585369,gender,gender
28677,younger sister,son,0.606430,0.586758,0.596094,gender,gender
28678,younger sister,uncle,0.596452,0.599123,0.597306,gender,gender


In [128]:
merged_group = merged_group[[
    'category1', 'group1', 'category2', 'group2', 'P_score', 'R_score', 'F1_score']]

In [130]:
merged_group.to_csv('qwen_group_bert_score.csv',index = False)

### Baidu

In [131]:
group1_list = []
group2_list = []
response1_list = []
response2_list = []
# all the combinations for all the possible groups
for index, row in similarity_df.iterrows():
    print(index, end='\r')
    group1_responses = baidu[baidu['group_English'] ==
                            row['group1']]['response_values'].dropna().tolist()
    group2_responses = baidu[baidu['group_English'] ==
                            row['group2']]['response_values'].dropna().tolist()
    # Generate combinations of responses between the two groups
    for response1, response2 in itertools.product(group1_responses, group2_responses):
        group1_list.append(row['group1'])
        group2_list.append(row['group2'])
        response1_list.append(response1)
        response2_list.append(response2)

# Create the final DataFrame with all combinations
result_df = pd.DataFrame({
    'group1': group1_list,
    'group2': group2_list,
    'response1': response1_list,
    'response2': response2_list
})

In [132]:
merged_data = result_df.merge(baidu_uniqe_bert_score, left_on = ['response1', 'response2'], right_on= ['ones','twos'],how = 'left')

In [133]:
merged_data.drop(['ones', 'twos'], axis = 1,inplace = True)

In [134]:
merged_data

,group1,group2,response1,response2,P_score,R_score,F1_score
0,People from low-income families,People from impoverished families,多,穷,0.740600,0.740600,0.740600
1,People from low-income families,People from impoverished families,多,多,1.000000,1.000000,1.000000
2,People from low-income families,People from impoverished families,多,少,0.813802,0.813802,0.813802
3,People from low-income families,People from impoverished families,多,幸福,0.619811,0.567225,0.592353
4,People from low-income families,People from impoverished families,多,富有,0.628342,0.578975,0.602649
...,...,...,...,...,...,...,...
2271486,Non-Heterosexuals,People from Hebei,喜欢,穷,0.506389,0.540696,0.522981
2271487,Non-Heterosexuals,People from Hebei,喜欢,彪悍,0.622503,0.622503,0.622503
2271488,Non-Heterosexuals,People from Hebei,喜欢,墨迹,0.587648,0.571614,0.579520
2271489,Non-Heterosexuals,People from Hebei,喜欢,小个子,0.565229,0.542259,0.553506


In [135]:
# 使用 apply 方法和 lambda 函数更新 groups_similarity_baidu
merged_data['category1'] = merged_data['group1'].apply(
    lambda x: group_to_category.get(x, None))
merged_data['category2'] = merged_data['group2'].apply(
    lambda x: group_to_category.get(x, None))

In [13]:
merged_data

,group1,group2,response1,response2,P_score,R_score,F1_score
0,People from low-income families,People from impoverished families,多,穷,0.740600,0.740600,0.740600
1,People from low-income families,People from impoverished families,多,多,1.000000,1.000000,1.000000
2,People from low-income families,People from impoverished families,多,少,0.813802,0.813802,0.813802
3,People from low-income families,People from impoverished families,多,幸福,0.619811,0.567225,0.592353
4,People from low-income families,People from impoverished families,多,富有,0.628342,0.578975,0.602649
...,...,...,...,...,...,...,...
2271486,Non-Heterosexuals,People from Hebei,喜欢,穷,0.506389,0.540696,0.522981
2271487,Non-Heterosexuals,People from Hebei,喜欢,彪悍,0.622503,0.622503,0.622503
2271488,Non-Heterosexuals,People from Hebei,喜欢,墨迹,0.587648,0.571614,0.579520
2271489,Non-Heterosexuals,People from Hebei,喜欢,小个子,0.565229,0.542259,0.553506


In [136]:
# 按 group1 和 group2 分组，计算 P_score, R_score 和 F1_score 的平均值
merged_category = merged_data.groupby(['category1', 'category2'])[
    ['P_score', 'R_score', 'F1_score']].mean().reset_index()

In [137]:
merged_category

,category1,category2,P_score,R_score,F1_score
0,SES,SES,0.680462,0.677914,0.678741
1,SES,age,0.615417,0.593760,0.603502
2,SES,disability,0.643699,0.629940,0.636082
3,SES,disease,0.631331,0.618402,0.624267
4,SES,educational_qualification,0.650271,0.648901,0.649159
...,...,...,...,...,...
88,religion,region,0.609136,0.599767,0.603969
89,religion,religion,0.666733,0.652395,0.659089
90,religion,sexual_orientation,0.646462,0.634892,0.639965
91,sexual_orientation,region,0.598960,0.599755,0.598788


In [138]:
merged_category.to_csv('baidu_category_bert_score.csv',index = False)

In [139]:
merged_group = merged_data.groupby(['group1', 'group2'])[
    ['P_score', 'R_score', 'F1_score']].mean().reset_index()

In [140]:
merged_group

,group1,group2,P_score,R_score,F1_score
0,AIDS patient,Afghans,0.594182,0.617091,0.604700
1,AIDS patient,African Americans,0.592122,0.612634,0.601303
2,AIDS patient,Africans,0.578380,0.588141,0.582029
3,AIDS patient,Americans,0.590469,0.595746,0.591968
4,AIDS patient,Arabs,0.584150,0.584378,0.583248
...,...,...,...,...,...
25195,younger sister,mother,0.642781,0.640934,0.641160
25196,younger sister,older brother,0.635812,0.624614,0.629294
25197,younger sister,son,0.627218,0.602881,0.614139
25198,younger sister,uncle,0.644480,0.635128,0.639286


In [141]:
# 使用 apply 方法和 lambda 函数更新 groups_similarity_baidu
merged_group['category1'] = merged_group['group1'].apply(
    lambda x: group_to_category.get(x, None))
merged_group['category2'] = merged_group['group2'].apply(
    lambda x: group_to_category.get(x, None))

In [142]:
merged_group = merged_group[[
    'category1', 'group1', 'category2', 'group2', 'P_score', 'R_score', 'F1_score']]

In [144]:
merged_group.to_csv('baidu_group_bert_score.csv',index = False)